<a href="https://colab.research.google.com/github/aaronjoseph/NLP_Boiler_Plate/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import os

In [ ]:
# Data Cleaning Process
def preprocess(text):
  text=text.lower()
  # remove hyperlinks
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
  text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
  #remove hashtag sign
  text=re.sub(r"#","",text)   
  #remove mentions
  text = re.sub(r"(?:\@)\w+", '', text)
  #remove non ascii chars
  text=text.encode("ascii",errors="ignore").decode()
  #remove some puncts (except . ! ?)
  text=re.sub(r'[:"#$%&\*+,-/:;<=>@\\^_`{|}~]+','',text)
  text=re.sub(r'[!]+','!',text)
  text=re.sub(r'[?]+','?',text)
  text=re.sub(r'[.]+','.',text)
  text=re.sub(r"'","",text)
  text=re.sub(r"\(","",text)
  text=re.sub(r"\)","",text)    
  text=" ".join(text.split())
  return text

